In [1]:
import pandas as pd
import math
from collections import namedtuple, defaultdict

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
Point = namedtuple("Point", ['x', 'y'])

In [4]:
def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

In [5]:
def solve_it(input_data, time_limit=120, file_name='LOG'):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    node_count = int(lines[0])

    points = []
    for i in range(1, node_count+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
    
    #Distance matrix
    distance_matrix = defaultdict(dict)
    for i, p_i in enumerate(points):
        for j, p_j in enumerate(points):
            distance_matrix[i][j] = int(length(p_i, p_j))
    
    manager = pywrapcp.RoutingIndexManager(node_count, 1, 0)  # 1-vehicle, 0-depot
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = time_limit
    search_parameters.log_search = True

    # Solve the problem.
    result = routing.SolveWithParameters(search_parameters)

    print(file_name+": OBJECTIVE VALUE IS ", result.ObjectiveValue())    
        
    solution=[]

    index = routing.Start(0)
    while not routing.IsEnd(index):
        solution.append(manager.IndexToNode(index))
        previous_index = index
        index = result.Value(routing.NextVar(index))
    
    return solution

In [6]:
def get_result(file_name, time_limit=120):
    with open('data/'+file_name, 'r') as input_data_file:
        input_data = input_data_file.read()
    solution = solve_it(input_data, time_limit, file_name)
    res = pd.DataFrame(solution, columns=['Order'])
    res.to_excel(file_name+'_res.xlsx', index=False, header=True)

In [ ]:
for file_name in ['1_tsp_51_1', '2_tsp_100_3', '3_tsp_200_2', '4_tsp_574_1', '5_tsp_1889_1', '6_tsp_33810_1']:
    get_result(file_name, 20)

1_tsp_51_1: OBJECTIVE VALUE IS  0


In [7]:
def get_precise_result(file_name, required_sol):
    with open('data/'+file_name, 'r') as input_data_file:
        input_data = input_data_file.read()
    solution = solve_it_with_other_params(input_data, required_sol, file_name)
    res = pd.DataFrame(solution, columns=['Order'])
    res.to_excel(file_name+'_res.xlsx', index=False, header=True)

In [8]:
def solve_it_with_other_params(input_data, required_solution, file_name='LOG'):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    node_count = int(lines[0])

    points = []
    for i in range(1, node_count+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
    
    #Distance matrix
    distance_matrix = defaultdict(dict)
    distance_matrix_float = defaultdict(dict)
    for i, p_i in enumerate(points):
        for j, p_j in enumerate(points):
            distance_matrix[i][j] = int(length(p_i, p_j))
            distance_matrix_float[i][j] = length(p_i, p_j)
    
    manager = pywrapcp.RoutingIndexManager(node_count, 1, 0)  # 1-vehicle, 0-depot
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    def solution_callback():
        cost = 0
        index = routing.Start(0)
        while not routing.IsEnd(index):
            previous_index = index
            index = routing.NextVar(index).Value()
            cost += distance_matrix_float[manager.IndexToNode(previous_index)][manager.IndexToNode(index)]
        first = routing.Start(0)
        cost += distance_matrix_float[manager.IndexToNode(index)][manager.IndexToNode(first)]
        if cost != 0 and cost < required_solution:
            print('PERFECT SOLUTION FOUND')
            routing.solver().FinishCurrentSearch()
        
    
    routing.AddAtSolutionCallback(solution_callback)
    
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    #search_parameters.time_limit.seconds = 20
    search_parameters.log_search = True

    # Solve the problem.
    result = routing.SolveWithParameters(search_parameters)

    print(file_name+": OBJECTIVE VALUE IS ", result.ObjectiveValue())    
        
    solution=[]

    index = routing.Start(0)
    while not routing.IsEnd(index):
        solution.append(manager.IndexToNode(index))
        previous_index = index
        index = result.Value(routing.NextVar(index))
    
    return solution

In [19]:
get_precise_result('1_tsp_51_1', 429)

PERFECT SOLUTION FOUND
1_tsp_51_1: OBJECTIVE VALUE IS  412


In [17]:
get_precise_result('4_tsp_574_1', 37600)

4_tsp_574_1: OBJECTIVE VALUE IS  0


In [ ]:
get_precise_result('5_tsp_1889_1', 323000)

In [65]:
def solve_it_for_last(input_data, required_solution, file_name='LOG'):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    node_count = int(lines[0])

    points = []
    for i in range(1, node_count+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
    
    manager = pywrapcp.RoutingIndexManager(node_count, 1, 0)  # 1-vehicle, 0-depot
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(length(points[from_node], points[to_node]))
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    class Solution_Callback():
        def __init__(self):
            self.best_value = float('inf')
        def __call__(self):
            if routing.CostVar().Min() >= self.best_value:
                return
            self.best_value = routing.CostVar().Min()
            cost = 0
            index = routing.Start(0)
            while not routing.IsEnd(index):
                previous_index = index
                index = routing.NextVar(index).Value()
                cost += length(points[manager.IndexToNode(previous_index)], points[manager.IndexToNode(index)])
            first = routing.Start(0)
            cost += length(points[manager.IndexToNode(index)], points[manager.IndexToNode(first)])
            if cost != 0 and cost < required_solution:
                print('PERFECT SOLUTION FOUND')
                routing.solver().FinishCurrentSearch()
            
    solution_callback = Solution_Callback()   
    
    routing.AddAtSolutionCallback(solution_callback)
    
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    #search_parameters.time_limit.seconds = 20
    search_parameters.log_search = True

    # Solve the problem.
    result = routing.SolveWithParameters(search_parameters)

    print(file_name+": OBJECTIVE VALUE IS ", result.ObjectiveValue())    
        
    solution=[]

    index = routing.Start(0)
    while not routing.IsEnd(index):
        solution.append(manager.IndexToNode(index))
        previous_index = index
        index = result.Value(routing.NextVar(index))
    
    return solution

In [66]:
def get_last_result(file_name, required_sol):
    with open('data/'+file_name, 'r') as input_data_file:
        input_data = input_data_file.read()
    solution = solve_it_for_last(input_data, required_sol, file_name)
    res = pd.DataFrame(solution, columns=['Order'])
    res.to_excel(file_name+'_res.xlsx', index=False, header=True)

In [67]:
get_last_result('1_tsp_51_1', 429)

PERFECT SOLUTION FOUND
1_tsp_51_1: OBJECTIVE VALUE IS  412


In [68]:
get_last_result('6_tsp_33810_1', 78478868)

PERFECT SOLUTION FOUND
6_tsp_33810_1: OBJECTIVE VALUE IS  78471731
